In [ ]:
%load_ext autoreload
%autoreload 2

# Mutations

How do we represent mutations computationally?
In SeqLike, we use the family of `Mutation` objects (`Substitution`, `Deletion`, and `Insertion`) as primitives,
as well as their `MutationSet` collection.
Later on in the notebook, we will show the APIs built on top of these primitive objects
that enable fluent sequence design workflows.

First off, let's see a few examples in action to get a feel for how it is used.

Here's a seqlike object:

In [ ]:
from seqlike.SeqLike import aaSeqLike, _add
from seqlike.Mutation import Mutation, Substitution, Deletion, Insertion

s1 = aaSeqLike("MKAILV")


And here's a `Substitution` object:

In [ ]:
sub1 = Substitution(position=3, mutant_letter="K")
sub1

They can be added together:

In [ ]:
s1 + sub1

For comparison:

In [ ]:
s1

Built-in validation of the WT sequence happens if the expected WT sequence is specified:

In [ ]:
sub_with_wt = Substitution(wt_letter="K", position=1, mutant_letter="R")
# s1 + sub_with_wt  # will raise an error.

In [ ]:
sub_with_wt = Substitution(wt_letter="K", position=2, mutant_letter="R")
s1 + sub_with_wt  # will NOT raise an error.

Here's an Insertion object:

In [ ]:
ins1 = Insertion(position=4, mutant_letter="D")


It, too, can be added to a seqlike:

In [ ]:
s1 + ins1

Finally, a `Deletion` object:

In [ ]:
del1 = Deletion(position=2)
del1

Note how we don't have to specify the letter. It's a special case substitution.

In [ ]:
s1 + del1

## Magical String Parsing

Mutation strings are much easier to type than always specifying `position=3, mutant_letter="K"`.
Let's see how they work.

First up, `Substitutions`:

In [ ]:
sub2 = Substitution("3K") # identical to sub1

s1 + sub2


And for comparison:

In [ ]:
s1 + sub1

Next up, `Insertions`:

In [ ]:
ins2 = Insertion("4D")  # same as ins1
s1 + ins2

And for comparison:

In [ ]:
s1 + ins1

Finally, `Deletions` can be handled with magical string parsing too.

In [ ]:
del2 = Deletion("2-")  # same as del1
s1 + del2

And for comparison:

In [ ]:
s1 + del1

Exclusively for `Deletion` objects, 
if you happen to accidentally include a substitution letter,
it will be ignored.

In [ ]:
# Deletions' mutation strings that don't have "-" at the end are handled magically:
del3 = Deletion("2R")
s1 + del3

Finally, if you really don't like the gap, you can always ungap the resulting SeqLike:

In [ ]:
(s1 + del3).ungap()

## Mutation Sets

MutationSets allow for collections of one or more mutations to be housed together.
For example, let's combine one of our substitutions and insertions together.

In [ ]:
sub1, ins1

In [ ]:
from seqlike.MutationSet import MutationSet 

ms1 = MutationSet([sub1, ins1])
ms1
# ms3 = ms1 + ms2

`MutationSet` objects have a special property that shows which positions are represented.

In [ ]:
ms1.positions

We can add a MutationSet to a SeqLike object.

In [ ]:
s1 + ms1

The operations are don't modify internal state, 
so re-running them again guarantees identical results:

In [ ]:
s1 + ms1

In [ ]:
s1 + ms1

Mutations in a MutationSet are applied from left to right.
What happens, though, if an Insertion (which changes the indexing),
is added before a Substitution (which doesn't)?

In [ ]:
sub1

In [ ]:
ms1_swapped = MutationSet([ins1, sub1])
ms1_swapped
s1 + ms1_swapped

Within a MutationSet, we preserve indexing w.r.t. the original sequence,
and internally propagate insertions of positions throughout the sequence.
If you have multiple MutationSets, however, 
then indexing is preserved w.r.t. the previous SeqLike, from left to right:

In [ ]:
# THIS:
s1 + ms1 + ms1_swapped

In [ ]:
# IS EQUIVALENT TO THIS:
intermediate = s1 + ms1
s2 = intermediate + ms1_swapped
s2

MutationSets are sets because only _unique_ positions can be represented.

In [ ]:
MutationSet(mutations="2A;3C".split(";")) + MutationSet(mutations="2D;4K".split(";"))

## Magical Mutation Set string parsing

It's really tedious to specify multiple mutations as specific objects,
so we have a magical parser that allows us to parse mutation strings:

In [ ]:
s1

In [ ]:
mutations = "^2A;^4D;5-" # insertion, substitution, deletion
ms2 = MutationSet(mutations=mutations.split(";"))

# The rest of the mutations in the set are offset by the correct amount
s1 + ms2

In [ ]:
import pandas as pd 

series = pd.Series(["2A;3C;4D", "2A;3C"], name="mutations")
series.apply(lambda mutation_str: MutationSet(mutation_str.split(";"))).apply(lambda mutset: s1 + mutset)

In [ ]:
# Get back ;-delimited string:
str(ms1)

In [ ]:
ms1.to_str()

## Mutational Scanning

In [ ]:
from seqlike import SeqLike 
from typing import List 

def alanine_scan(s: SeqLike) -> List[SeqLike]:
    mutants = []
    for i in range(len(s)):
        mutants.append(s + Substitution(f"{i}A"))
    return mutants


We've wrapped that functionality in the SeqLike class.

In [ ]:
s1.scan("A")


In [ ]:
s1.scan("W")


In [ ]:
# Do an Alanine Scan but ensure wanted mutations w.r.t. WT are preserved.
s1.scan("A").apply(lambda seq: seq + MutationSet("1M;6C".split(";")))


## Differencing SeqLikes

The `__sub__` operator has been overloaded such that if we subtract one seqlike from another seqlike,
we get back a mutation set w.r.t. the left seqlike that can be added back to the left seqlike to obtain the right seqlike.

For example, with a SeqLikes `s1`:

In [ ]:
s1


And a particular mutation:

In [ ]:
sub1

We can obtain the difference between `s1` and `s1 + ms1`:

In [ ]:
diff1 = s1 - (s1 + sub1)
diff1

The resulting MutationSet is an _inferred_ set of mutations needed
to reconstruct the sequence on the right side of the plus sign from the left side.
It may not always be the same as the original mutation set.
Numbering is always going to be with respect to an ungapped reference (left hand side) sequence.

In [ ]:
diff2 = (s1 + sub1) - s1
diff2

We can apply the mutation inferred mutation sets and verify that we get back the same mutated sequence:

In [ ]:
s1 + sub1

which can be compared to:

In [ ]:
s1 + diff1

In [ ]:
(s1 + sub1).to_str() ==  (s1 + diff1).to_str()

Let's try with a mutation set, one that is a bit more complicated:

In [ ]:
ms1

In [ ]:
diff1 = s1 - (s1 + ms1)
diff1

In [ ]:
s1 + ms1

In [ ]:
s1 + diff1

In [ ]:
(s1 + ms1).to_str() == (s1 + diff1).to_str()


In [ ]:
s1

In [ ]:
(s1 + ms2).ungap()

In [ ]:
ms1

In [ ]:
ms3 = MutationSet("2A;3F;4Q".split(";"))

In [ ]:
s1 + ms3

## Prototype Replacement? Using difflib

Below is an early prototype we built using the `difflib` built-in library in Python.
As you can see, handling mutations with the desired semantics is difficult in the absence of a pairwise alignment,
but doing so may actually be faster than calling out to MAFFT.
We leave it below here in the interest of transparency
and to possibly spark ideas on modifications that may make it work!

In [ ]:
## PROTOTYPE

import difflib

diff_indels = []

for i, d in enumerate(difflib.ndiff(s1.to_str(), (s1 + ms3).ungap().to_str())):
    print(d)
    if d[0] == "-":
        diff_indels.append(Deletion(f"{i + 1}-"))
    elif d[0] == "+":
        diff_indels.append(Insertion(f"^{i + 1}{d[-1]}"))
diff_indels

In [ ]:
ms3

In [ ]:
from copy import deepcopy 

def coalesce(diff_indels):
    original_diff_indels = deepcopy(diff_indels)

    coalesced_mutations = []
    offset = 0
    while original_diff_indels:
        if len(original_diff_indels) == 1:
            coalesced_mutations.append(original_diff_indels[0] - offset)
            break 
        m1, m2 = original_diff_indels[0:2]
        # A substitution is always represented in in ndiff as a Deletion followed by an Insertion right after.
        if isinstance(m1, Deletion) and isinstance(m2, Insertion) and m1.position + 1 == m2.position:
            coalesced_mutations.append(Substitution(f"{m1.position}{m2.mutant_letter}") - offset)
            original_diff_indels.remove(m1)
            original_diff_indels.remove(m2)
            offset += 1
        else:
            coalesced_mutations.append(m1 - offset)
            original_diff_indels.remove(m1)
            offset += 1
    return MutationSet(coalesced_mutations)

ms3_recon = coalesce(diff_indels)
ms3_recon

In [ ]:

s1 + ms3

In [ ]:
s1 + ms3_recon